In [50]:
import json
import numpy as np
import pandas as pd
import os 
from os import walk
import shutil
import splitfolders

In [31]:
!pip install split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# **From Kaggle**

In [2]:
from getpass import getpass
import os

user='mohandabdelhalim'
key="8486e9c87e92ff7f979667dd05275afa"


if '.kaggle' not in os.listdir('/root'):
       !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json


!kaggle datasets download -d danielbacioiu/tig-aluminium-5083
!chmod 777 *

100% 11.2G/11.2G [01:28<00:00, 176MB/s]
100% 11.2G/11.2G [01:28<00:00, 136MB/s]


In [3]:
%%capture
!unzip /content/tig-aluminium-5083.zip

# **Labels Encoding**

In [4]:
class_names=['0_Good weld','1_Defect']
for i in class_names:
      os.makedirs(os.path.join('all_labeled',i))

# **# Load Train imgs & labels**

In [5]:
import json
f=open("/content/al5083/train/train.json",)
train_labels=json.load(f)

In [6]:
train_labels['170815-133921-Al 2mm/frame_00080.png']

2

#**Generating dict of subfolders and data contained as documented**

In [7]:
def Data_collected_dir(typ_data=train_labels):
    keys=list(typ_data.keys())
    vals=list(typ_data.values())
    k_keys=[]
    for k in keys :
      a_string=k.replace('/',' ')
      x=a_string.split()
      string=x[0]+' '+x[1]
      k_keys.append(string)
    true_labels_dict = {k_keys[i]: vals[i] for i in range(len(k_keys))} 

    return true_labels_dict


In [8]:
true_labels_dict=Data_collected_dir()

In [9]:
def create_data_csv(location_folders,typ_data_dict=true_labels_dict):
  folders=[]
  for (_,dirnames,_) in walk(location_folders):
    folders.extend(dirnames)
  num_files=[]
  for folder in folders:
      location_files=location_folders+folder+'/'
      files=[]
      for (_,_,file_name) in walk(location_files):
        files.extend(file_name)
        break
      num_files.append(len(files))
  labels=[typ_data_dict[folder]  for folder in folders]
  Data=pd.DataFrame({'folder':folders,'num_files':np.array(num_files),'label':labels})
  return Data.sort_values(by='label')

In [10]:
train_data_csv=create_data_csv('/content/al5083/al5083/train/')

In [11]:
train_data_csv['label'] = train_data_csv['label'] .replace([1,2,3,4,5],1) 

In [12]:
train_data_csv

,folder,num_files,label
22,170906-120346-Al 2mm,1165,0
40,170913-151508-Al 2mm-part1,371,0
35,170913-142501-Al 2mm,770,0
29,170905-115602-Al 2mm,914,0
25,170913-143933-Al 2mm-part1,379,0
24,170906-114912-Al 2mm,979,0
19,170913-152931-Al 2mm-part1,354,0
16,170906-141809-Al 2mm-part1,1037,0
13,170913-155806-Al 2mm-part1,932,0
12,170904-141730-Al 2mm-part1,394,0


In [52]:
train_data_csv.to_csv("train_data_csv.csv")

**List_data_to_folders_per_label**

In [13]:
def copy_to_folder_2classes(num, raw_data_folder ,data_labeled  ,new_folder_path ):
  #test_files='/content/al5083/test'
  for folder_n in list(data_labeled[data_labeled['label']== num]['folder']):
    #print(num,folder_n)

    for file in os.listdir(raw_data_folder):    
      location_files = os.path.join(raw_data_folder,folder_n)

      for (_, _, filenames) in walk(location_files):
          # print(filenames)  # lists of string contain images names 'frame_00813.png'

        for image in filenames:
          get_image = os.path.join(location_files,image)
          # print(get_image)  #/content/al5083/test/170906-143512-Al 2mm-part1/frame_01214.png

          new_image_name = folder_n + image
          shutil.move(get_image,os.path.join(location_files,new_image_name))
          shutil.move(os.path.join(location_files,new_image_name), new_folder_path + class_names[num])


In [14]:
location_folders='/content/al5083/train/'
for i in range(len(class_names)):
   copy_to_folder_2classes(i, location_folders , train_data_csv , '/content/all_labeled/')   

In [15]:
len(os.listdir('/content/all_labeled/0_Good weld'))

8758

In [16]:
len(os.listdir('/content/all_labeled/1_Defect'))

17908

In [23]:
test_json = open('/content/al5083/test/test.json') 
test_labels = json.load(test_json) 

In [24]:
test_labels_dict=Data_collected_dir(typ_data=test_labels)

In [25]:
test_data_csv=create_data_csv('/content/al5083/test/',typ_data_dict=test_labels_dict)

In [26]:
test_data_csv['label'] = test_data_csv['label'] .replace([1,2,3,4,5],1) 

In [27]:
test_data_csv

,folder,num_files,label
0,170913-154448-Al 2mm,427,0
2,170905-112213-Al 2mm,585,0
12,170906-113317-Al 2mm-part1,526,0
13,170906-104925-Al 2mm,651,0
4,170906-143512-Al 2mm-part2,351,1
1,170904-155610-Al 2mm,235,1
3,170904-154202-Al 2mm-part1,631,1
5,170906-143512-Al 2mm-part1,1212,1
6,170906-155007-Al 2mm-part1,451,1
10,170906-151724-Al 2mm-part1,402,1


In [28]:
test_location_folders='/content/al5083/test/'
for i in range(len(class_names)):
   copy_to_folder_2classes(i, test_location_folders , test_data_csv  , '/content/all_labeled/')   

In [53]:
test_data_csv.to_csv("test_data_csv.csv")

**2 Classes size**

In [29]:
len(os.listdir('/content/all_labeled/0_Good weld'))

10947

In [30]:
len(os.listdir('/content/all_labeled/1_Defect'))

22307

**Train Test Split**

In [37]:
input_folder = "/content/all_labeled" #Enter Input Folder
output = "/content/" #Enter Output Folder

splitfolders.ratio(input_folder, output=output, seed=42, ratio=(0.7,0.15,0.15))

Copying files: 33254 files [01:52, 295.80 files/s]


In [38]:
print('train size good weld {}'.format(len(os.listdir('/content/train/0_Good weld'))))
print('train size Defect {}'.format(len(os.listdir('/content/train/1_Defect'))))

train size good weld 7662
train size Defect 15614


In [39]:
print('val size good weld {}'.format(len(os.listdir('/content/val/0_Good weld'))))
print('val size Defect {}'.format(len(os.listdir('/content/val/1_Defect'))))

val size good weld 1642
val size Defect 3346


In [40]:
print('test size good weld {}'.format(len(os.listdir('/content/test/0_Good weld'))))
print('test size Defect {}'.format(len(os.listdir('/content/test/1_Defect'))))

test size good weld 1643
test size Defect 3347


**Move to Drive**

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
!mv        '/content/train'           '/content/drive/MyDrive/Defect_inspection/Train'

In [44]:
!mv        '/content/val'           '/content/drive/MyDrive/Defect_inspection/Val'
!mv        '/content/test'           '/content/drive/MyDrive/Defect_inspection/Test'

In [54]:
!mv   '/content/train_data_csv.csv'    '/content/drive/MyDrive/Defect_inspection/Train'
!mv   '/content/test_data_csv.csv'     '/content/drive/MyDrive/Defect_inspection/Test'